In [40]:
import csv
import pandas as pd
import keras
from skimage import io, transform

"""
ASSUMPTIONS :
DATA IS DOWNLOADED FROM KAGGLE AVITO AND STORED IN DIRECTORIES LIKE
/home/edwin/data/avito/Images_0/0/199300.jpg 
PLEASE CHANGE BASE_DIRECTORY to YOUR IMAGES DIRECTORY (ie /home/edwin/data/avito/Images_  )
              PAIRS_INFO_FILE to YOUR ITEM PAIRS FILE
              ITEM INTO FILES TO YOUR ITEM INFO FILE
"""

BASE_DIRECTORY = "/home/edwin/data/avito/Images_"  # location of stored kaggle images [0-9] Images_0,Images_1...
DELIMITER = "/"   # OS specific directory delimiters
PAIRS_INFO_FILE = "/home/edwin/data/avito/ItemPairs_train.csv" # location of pair information and truth
ITEM_INFO_FILE = "/home/edwin/data/avito/ItemInfo_train.csv" 

def pandas_read_file(file_name):
    df = pd.read_csv(file_name)
    return df

my_pairs = pandas_read_file(PAIRS_INFO_FILE)
print (my_pairs.shape)

(2991396, 4)


In [223]:
def get_image(image_index,base_directory,delimiter,file_type=".jpg"):
    """
    params : 
    image_index - kaggle image index 
    base_directory - location of image directories
    delimiter - depending on OS
    file_type - type of input image, default jpg
    
    will always import grayscale
    """
    image_index = str(image_index)
    if len(image_index) > 1 :
        index_1 = str(image_index)[-2]
    else :
        index_1 = str(0)
    index_2 = str(image_index)[-1]
    if index_1 != '0' :
        index_2 = index_1 + index_2
    image_location = BASE_DIRECTORY + str(index_1) + delimiter + index_2 + delimiter + image_index + file_type
    try :
        image = io.imread(image_location, as_grey=True)
    except :
        return 0
    return transform.resize(image, (64, 64))

print (get_image("1000",BASE_DIRECTORY,DELIMITER).shape)

(64, 64)


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [45]:
my_info_pairs = read_file(ITEM_INFO_FILE)
merged_pairs = my_compressed_pairs.merge(my_info_pairs, left_on='itemID_1', right_on="itemID", how='inner')
merged_pairs = merged_pairs.merge(my_info_pairs, left_on='itemID_2', right_on="itemID", how='inner')
reset_selective my_info_pairs

(300708, 26)


In [232]:
from itertools import product
import numpy as np

def generate_truth_labels(my_df):
    def generate_pairs(row):
        if isinstance(row["images_array_x"],float) :
            array_x = [str(row["images_array_x"]),]
        else :
            array_x = row["images_array_x"].split(",")
        array_x = [i for i in filter(lambda x: x != 'nan',array_x)]
        if isinstance(row["images_array_y"],float) :
            array_y = [str(row["images_array_y"]),]
        else :
            array_y = row["images_array_y"].split(",")
        array_y = [i for i in filter(lambda x: x != 'nan',array_y)]
        return [(x,y) for x,y in product(array_x,array_y)],row["isDuplicate"]
    return my_df.apply(lambda row : generate_pairs(row) ,axis=1)
my_image_df = generate_truth_labels(merged_pairs)


In [235]:
import math

def generate_X_y(my_df,lim_size):
    X = np.zeros(shape=(lim_size,2))
    y = np.zeros(shape=(lim_size,1))
    i = 0
    for j in range(len(my_df)):
        row = my_df[j]
        truth = row[1]
        for index_1, index_2 in row[0]:
            X[i][0] = index_1
            X[i][1] = index_2
            y[i][0] = truth
            i += 1
            if i >= lim_size :
                return X,y,j
    X = X[:i]
    y = y[:i]
    return X,y,j
X,y,j = generate_X_y(my_image_df,100000000)

print (X.shape,y.shape)

(2955686, 2) (2955686, 1)


In [290]:
def get_relevant_images(my_input):
    return [[get_image(int(line[0]),BASE_DIRECTORY,DELIMITER),get_image(int(line[1]),BASE_DIRECTORY,DELIMITER)] for line in my_input]

x_key = get_relevant_images(X[:1000])

def apply_filter(x,y):
    return list(filter(lambda k : k[0][0] is not None and k[0][1] is not None, zip(x,y)))

filtered_list = apply_filter(x_key,y)


/usr/local/lib/python3.5/dist-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [297]:
X_image = np.array([i[0] for i in filtered_list])
y_image = np.array([i[1] for i in filtered_list])

In [298]:
print (X_image, y_image)

[[array([[ 0.37314868,  0.57296781,  0.72473653, ...,  1.        ,
         1.        ,  1.        ],
       [ 0.40932171,  0.68244621,  0.70445563, ...,  1.        ,
         1.        ,  1.        ],
       [ 0.44256108,  0.48098998,  0.69882699, ...,  1.        ,
         1.        ,  1.        ],
       ..., 
       [ 0.41484916,  0.41662229,  0.432555  , ...,  0.34617759,
         0.33864833,  0.32310138],
       [ 0.4137376 ,  0.41239569,  0.42011627, ...,  0.3518091 ,
         0.34303581,  0.31533338],
       [ 0.40906843,  0.40823093,  0.4147021 , ...,  0.348294  ,
         0.33392793,  0.31579315]]), array([[ 0.93755255,  0.94453784,  0.94931725, ...,  0.94093137,
         0.93247549,  0.92781863],
       [ 0.93755255,  0.94453784,  0.94919949, ...,  0.93728075,
         0.93247549,  0.92781863],
       [ 0.936254  ,  0.94061789,  0.94498179, ...,  0.93321078,
         0.92889572,  0.92453182],
       ..., 
       [ 0.32503934,  0.32531765,  0.33021784, ...,  0.30421029,
     